In [19]:
%load_ext autoreload
%autoreload 2
import numpy as np
import ldpc.protograph as pt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Protograph class

## Ring of circulants

In [20]:
a=pt.ring_of_circulants_f2([1])
a

protograph.ring_of_circulants_f2((1))

In [21]:
a.T #Transpose

protograph.ring_of_circulants_f2((-1))

In [22]:
a+a.T #addition over the ring

protograph.ring_of_circulants_f2((1,-1))

In [23]:
a*a #multplication over the ring

protograph.ring_of_circulants_f2((2))

In [24]:
(a+a.T).to_binary(lift_parameter=3) #binary mapping

array([[0, 1, 1],
       [1, 0, 1],
       [1, 1, 0]])

In [25]:
a=pt.array([[[0,1]]])
a.T.to_binary(3)


array([[1, 0, 1],
       [1, 1, 0],
       [0, 1, 1]])

## Protograph

A protograph is a matrix where each element is from the ring of circulants

In [26]:
proto_a=pt.array([[(0,1),(0)]])
print(repr(proto_a))
print(proto_a)

protograph.array([[(0,1),(0)]])
[[(0,1) (0)]]


In [27]:
print(pt.zeros((3,4))) #zero element

[[() () () ()]
 [() () () ()]
 [() () () ()]]


In [28]:
print(pt.identity(4)) #identity element

[[(0) () () ()]
 [() (0) () ()]
 [() () (0) ()]
 [() () () (0)]]


In [29]:
a=pt.identity(3)
b=pt.zeros(3)
print(a@a) #idenity multiplicaiton
print(a@b) #zero multiplication

[[(0) () ()]
 [() (0) ()]
 [() () (0)]]
[[() () ()]
 [() () ()]
 [() () ()]]


In [30]:
print(pt.kron(a,a))

[[(0) () () () () () () () ()]
 [() (0) () () () () () () ()]
 [() () (0) () () () () () ()]
 [() () () (0) () () () () ()]
 [() () () () (0) () () () ()]
 [() () () () () (0) () () ()]
 [() () () () () () (0) () ()]
 [() () () () () () () (0) ()]
 [() () () () () () () () (0)]]


In [31]:
print(pt.kron(a,b))

[[() () () () () () () () ()]
 [() () () () () () () () ()]
 [() () () () () () () () ()]
 [() () () () () () () () ()]
 [() () () () () () () () ()]
 [() () () () () () () () ()]
 [() () () () () () () () ()]
 [() () () () () () () () ()]
 [() () () () () () () () ()]]


In [32]:
a=pt.zeros(2)
repr(a)
a=pt.array([[(3),(2)],[(4),(3)]])
print(a.T) #transpose

[[(-3) (-4)]
 [(-2) (-3)]]


In [33]:
print(pt.kron(a,a.T,a.T))

[[(-3) (-4) (-4) (-5) (-4) (-5) (-5) (-6)]
 [(-2) (-3) (-3) (-4) (-3) (-4) (-4) (-5)]
 [(-2) (-3) (-3) (-4) (-3) (-4) (-4) (-5)]
 [(-1) (-2) (-2) (-3) (-2) (-3) (-3) (-4)]
 [(-2) (-3) (-3) (-4) (-3) (-4) (-4) (-5)]
 [(-1) (-2) (-2) (-3) (-2) (-3) (-3) (-4)]
 [(-1) (-2) (-2) (-3) (-2) (-3) (-3) (-4)]
 [(0) (-1) (-1) (-2) (-1) (-2) (-2) (-3)]]


## Lifted product codes

In [34]:
from bposd.css import css_code

def I(n):
    return pt.identity(n)

class lifted_hgp(css_code):

    def __init__(self,lift_parameter,a,b=None):

        '''
        Generates the lifted hypergraph product of the protographs a and b
        '''
        self.a=a

        self.a_m,self.a_n=self.a.shape

        if b is None:
            self.b=pt.copy(self.a)
        else:
            self.b=b
        
        self.b_m,self.b_n=self.b.shape

        self.hx1_proto=pt.kron(self.a,I(self.b_n))
        self.hx2_proto=pt.kron(I(self.a_m),self.b.T)
        self.hx_proto=pt.hstack([self.hx1_proto,self.hx2_proto])

        self.hz1_proto=pt.kron(I(self.a_n),self.b)
        self.hz2_proto=pt.kron(self.a.T,I(self.b_m))
        self.hz_proto=pt.hstack([self.hz1_proto,self.hz2_proto])

        super().__init__(self.hx_proto.to_binary(lift_parameter),self.hz_proto.to_binary(lift_parameter))

In [35]:
from lifted_hgp import lifted_hgp

d=21

a=pt.array([[(0,1)]])
b=pt.array([[(0,d)]])

qcode=lifted_hgp(d*(d-1),a,b)

qcode.test()

<Unnamed CSS code>, (2,4)-[[840,2,nan]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (2,4)-[[840,2,nan]]


True

In [36]:
def lpc1():
    proto_a=pt.array([
        [(0), (11), (7), (12)],
        [(1), (8), (1), (8)],
        [(11), (0), (4), (8)],
        [(6), (2), (4), (12)]
    ])
    return lifted_hgp(13,proto_a)

qcode=lpc1()
qcode.canonical_logicals()
qcode.test()

#Test to see whether the PCMS are the same as with the old version of the code
hx=np.loadtxt("hx.txt").astype(int)
hz=np.loadtxt("hz.txt").astype(int)
assert np.array_equal(qcode.hx,hx)
assert np.array_equal(qcode.hz,hz)

<Unnamed CSS code>, (4,8)-[[416,18,nan]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (4,8)-[[416,18,nan]]
